In [45]:
import pandas as pd
df = pd.read_parquet('batch_user_model_features.parquet', engine='pyarrow')

In [46]:
df

,recentSearches,recentlyViewedListingTitles,recentlyViewedListingTaxos,tfIdf_recentClickTfIdfVector,tfIdf_recentFavoriteTfIdfVector,tfIdf_recentCartTfIdfVector,user_id,meta_user_id
0,"[meow, woof, oink]","[cat costume, dog costume, pig costume, turkey...","[clothing.costumes.animals.cats, clothing.cost...",[hi],[bye],[woof],1,1
1,[],"[cat costume, dog costume, pig costume, turkey...","[clothing.costumes.animals.cats, clothing.cost...",[hi],[],[woof],2,2
2,"[meow, woof, oink, caw, cluck, tingtingting, d...","[cat costume, dog costume, pig costume, turkey...","[clothing.costumes.animals.cats, clothing.cost...",[hi],[bye],[woof],3,3


How are these features defined
https://github.com/etsy/recs/blob/a9ab6bc6feb9d09195451c2699605c535278d25f/models/candidate_com_u2l_dual_encoder/versions/v4/package_model_custom_seeds.py#L20

Example: https://github.com/etsy/recs/blob/a9ab6bc6feb9d09195451c2699605c535278d25f/models/candidate_com_u2l_dual_encoder/online_inference_testing/compare_offline_to_online_inference/make_online_request_custom_seed.py#L11

In [41]:
import tensorflow as tf
import tensorflow_transform as tft
import requests
import base64

In [28]:
user_model_path = "gs://etsy-recsys-ml-prod-data-nxsn/candidates/google_ann_online_deployments/u2ldev4-recsret-647-flop/2/"
tft_path = "gs://etsy-recsys-ml-prod-data-nxsn/candidates/u2l/dual_encoder/v4/latest_prod_model/transform_fn"

In [29]:
user_model = tf.saved_model.load(user_model_path)
tft_output = tft.TFTransformOutput(tft_path)

In [47]:
tft_output.raw_feature_spec()

{'activeListingBasics_daysSinceCreate': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'activeListingBasics_daysSinceOriginalCreate': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'activeListingBasics_normalizedDailyGms': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'activeListingBasics_normalizedDailyQuantity': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'activeListingBasics_pastDayOrders': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'activeListingBasics_totalFavoriteCount': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'activeListingBasics_totalGms': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'activeListingBasics_totalOrders': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'activeListingBasics_totalQuantitySold': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'activeListingShippingCosts_domesticShipCost': FixedLen

In [53]:
tft_output.raw_metadata.schema

feature {
  name: "activeListingBasics_daysSinceCreate"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "activeListingBasics_daysSinceOriginalCreate"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "activeListingBasics_normalizedDailyGms"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "activeListingBasics_normalizedDailyQuantity"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "activeListingBasics_pastDayOrders"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "activeListingBasics_totalFavoriteCount"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "activeListingBasics_totalGms"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "activeListingBasics_totalOrders"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
f

In [52]:
tft_output.transformed_feature_spec()

{'label': FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
 'listing_float_features': FixedLenFeature(shape=[27], dtype=tf.float32, default_value=None),
 'listing_string_features': FixedLenFeature(shape=[3], dtype=tf.string, default_value=None),
 'meta_listing_id': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'meta_user_id': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'user_string_features': FixedLenFeature(shape=[38], dtype=tf.string, default_value=None)}

In [30]:
user_model.signatures

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, examples) at 0x16B45FC70>, 'serving_complete': <ConcreteFunction signature_wrapper(*, examples) at 0x16B916E80>})

Signature definition: https://github.com/etsy/recs/blob/d16036f14bbca5f432904c888aaac042d4ddbddc/models/candidate_com_u2l_dual_encoder/model/model_io_utils.py#L142

In [44]:
feature_samples = [
    {
        "contextualInfo[name=user].rivuletUserInfo.timeseries.recentlySearchedQueries50FV1#searchQuery": [
            "meow",
            "woof",
            "oink",
        ],
        "contextualInfo[name=listing,nameIndex=0].docInfo.listingInfo.verticaListings.title": [
            "cat costume"
        ],
        "contextualInfo[name=listing,nameIndex=1].docInfo.listingInfo.verticaListings.title": [
            "dog costume"
        ],
        "contextualInfo[name=listing,nameIndex=2].docInfo.listingInfo.verticaListings.title": [
            "pig costume"
        ],
        "contextualInfo[name=listing,nameIndex=3].docInfo.listingInfo.verticaListings.title": [
            "turkey costume"
        ],
        "contextualInfo[name=listing,nameIndex=0].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.cats"
        ],
        "contextualInfo[name=listing,nameIndex=1].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.dogs"
        ],
        "contextualInfo[name=listing,nameIndex=2].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.pigs"
        ],
        "contextualInfo[name=listing,nameIndex=3].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.turkeys"
        ],
        "contextualInfo[name=user].userInfo.tfIdf.recentClickTfIdfVector#keys": ["hi"],
        "contextualInfo[name=user].userInfo.tfIdf.recentFavoriteTfIdfVector#keys": ["bye"],
        "contextualInfo[name=user].userInfo.tfIdf.recentCartTfIdfVector#keys": ["woof"],
    },
    {
        "contextualInfo[name=user].rivuletUserInfo.timeseries.recentlySearchedQueries50FV1#searchQuery": [],
        "contextualInfo[name=listing,nameIndex=0].docInfo.listingInfo.verticaListings.title": [
            "cat costume"
        ],
        "contextualInfo[name=listing,nameIndex=1].docInfo.listingInfo.verticaListings.title": [
            "dog costume"
        ],
        "contextualInfo[name=listing,nameIndex=2].docInfo.listingInfo.verticaListings.title": [
            "pig costume"
        ],
        "contextualInfo[name=listing,nameIndex=3].docInfo.listingInfo.verticaListings.title": [
            "turkey costume"
        ],
        "contextualInfo[name=listing,nameIndex=0].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.cats"
        ],
        "contextualInfo[name=listing,nameIndex=1].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.dogs"
        ],
        "contextualInfo[name=listing,nameIndex=2].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.pigs"
        ],
        "contextualInfo[name=listing,nameIndex=3].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.turkeys"
        ],
        "contextualInfo[name=user].userInfo.tfIdf.recentClickTfIdfVector#keys": ["hi"],
        "contextualInfo[name=user].userInfo.tfIdf.recentFavoriteTfIdfVector#keys": [],
        "contextualInfo[name=user].userInfo.tfIdf.recentCartTfIdfVector#keys": ["woof"],
    },
    {
        "contextualInfo[name=user].rivuletUserInfo.timeseries.recentlySearchedQueries50FV1#searchQuery": [
            "meow",
            "woof",
            "oink",
            "caw",
            "cluck",
            "tingtingting",
            "dingeringeding",
            "Hatee",
            "Hatee",
            "Hatee",
            "ho",
        ],
        "contextualInfo[name=listing,nameIndex=0].docInfo.listingInfo.verticaListings.title": [
            "cat costume"
        ],
        "contextualInfo[name=listing,nameIndex=1].docInfo.listingInfo.verticaListings.title": [
            "dog costume"
        ],
        "contextualInfo[name=listing,nameIndex=2].docInfo.listingInfo.verticaListings.title": [
            "pig costume"
        ],
        "contextualInfo[name=listing,nameIndex=3].docInfo.listingInfo.verticaListings.title": [
            "turkey costume"
        ],
        "contextualInfo[name=listing,nameIndex=4].docInfo.listingInfo.verticaListings.title": [
            "goat costume"
        ],
        "contextualInfo[name=listing,nameIndex=0].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.cats"
        ],
        "contextualInfo[name=listing,nameIndex=1].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.dogs"
        ],
        "contextualInfo[name=listing,nameIndex=2].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.pigs"
        ],
        "contextualInfo[name=listing,nameIndex=3].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.turkeys"
        ],
        "contextualInfo[name=listing,nameIndex=4].docInfo.listingInfo.verticaListings.taxonomyPath": [
            "clothing.costumes.animals.goat"
        ],
        "contextualInfo[name=user].userInfo.tfIdf.recentClickTfIdfVector#keys": ["hi"],
        "contextualInfo[name=user].userInfo.tfIdf.recentFavoriteTfIdfVector#keys": ["bye"],
        "contextualInfo[name=user].userInfo.tfIdf.recentCartTfIdfVector#keys": ["woof"],
    },
]

for STRING_LIST_FEATURE_VALUES in feature_samples:
    features = {
        name: tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[a.encode("utf-8") for a in values])
        )
        for (name, values) in STRING_LIST_FEATURE_VALUES.items()
    }

    example = tf.train.Example(features=tf.train.Features(feature=features))
    output = user_model(example)


TypeError: Could not generate a generic TraceType for features {
  feature {
    key: "contextualInfo[name=listing,nameIndex=0].docInfo.listingInfo.verticaListings.taxonomyPath"
    value {
      bytes_list {
        value: "clothing.costumes.animals.cats"
      }
    }
  }
  feature {
    key: "contextualInfo[name=listing,nameIndex=0].docInfo.listingInfo.verticaListings.title"
    value {
      bytes_list {
        value: "cat costume"
      }
    }
  }
  feature {
    key: "contextualInfo[name=listing,nameIndex=1].docInfo.listingInfo.verticaListings.taxonomyPath"
    value {
      bytes_list {
        value: "clothing.costumes.animals.dogs"
      }
    }
  }
  feature {
    key: "contextualInfo[name=listing,nameIndex=1].docInfo.listingInfo.verticaListings.title"
    value {
      bytes_list {
        value: "dog costume"
      }
    }
  }
  feature {
    key: "contextualInfo[name=listing,nameIndex=2].docInfo.listingInfo.verticaListings.taxonomyPath"
    value {
      bytes_list {
   

In [35]:
parsed_features = tf.io.parse_example(example, feature_spec)

In [36]:
parsed_features

{'listingId': <tf.Tensor: shape=(), dtype=int64, numpy=1192518648>,
 'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=692898591>}

In [37]:
output = user_model(example)

ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * (b'\n.\n\x14\n\x07user_id\x12\t\x1a\x07\n\x05\x9f\x96\xb3\xca\x02\n\x16\n\tlist'
 b'ingId\x12\t\x1a\x07\n\x05\xf8\xc7\xd1\xb8\x04')
    * False
    * None
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * {'user_string_features': TensorSpec(shape=(None, 38), dtype=tf.string, name='user_string_features')}
    * False
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * {'user_string_features': TensorSpec(shape=(None, 38), dtype=tf.string, name='inputs_user_string_features')}
    * False
    * None
  Keyword arguments: {}

Option 3:
  Positional arguments (3 total):
    * {'user_string_features': TensorSpec(shape=(None, 38), dtype=tf.string, name='inputs_user_string_features')}
    * True
    * None
  Keyword arguments: {}

Optio